In [1]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('C:/Users/82102/res/dont_sleep/converted_keras/keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('C:/Users/82102/res/dont_sleep/test_photo.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)

C:\Users\82102\AppData\Local\Temp\ipykernel_6524\3729294492.py:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = ImageOps.fit(image, size, Image.ANTIALIAS)


1/1 [==============================] - 1s 764ms/step
[[0.999954   0.00004596]]


In [ ]:
import cv2
import beepy
import tensorflow.keras
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import QTimer, Qt
from PyQt5 import QtGui

# 컴퓨터에 내장된 소리를 출력
def beepsound():
    beepy.beep(sound=6)

# image preprocessing
def preprocessing(frame):
    # Resize
    size = (224, 224)
    frame_resized = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)

    # normalize image
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1

    # Reshape image dimensions - reshape them for prediction.
    frame_reshaped = frame_normalized.reshape((1, 224, 224, 3))

    return frame_reshaped

## Load the trained model
model_filename = 'C:/Users/82102/res/dont_sleep/converted_keras/keras_model.h5'
model = tensorflow.keras.models.load_model(model_filename)

class MainWindow(QMainWindow):
    sleep_cnt = 1 # 30초간 "졸림" 상태를 확인하기 위한 변수
    def __init__(self):
        super().__init__()
        
        self.text_label = QLabel(self)
        self.text_label.setAlignment(Qt.AlignCenter)
        
        # Create a label to display the webcam stream
        self.label = QLabel(self)
        self.label.setAlignment(Qt.AlignCenter)

        # Create a layout for the label
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.text_label)
        
        self.text_label.setText("Some text")

        # Create a central widget and set the layout
        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # camera capture object, 0=internal camera, 1=external camera
        self.capture = cv2.VideoCapture(0)

        # Adjust capture frame size
        self.capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
        self.capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

        # Create a timer to update the label with the webcam stream
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(50)

    def update_frame(self):
        
        ret, frame = self.capture.read()

        if ret:
            # flip image
            frame_flipped = cv2.flip(frame, 1)

            # data preprocessing
            preprocessed = preprocessing(frame_flipped)

            # prediction
            prediction = model.predict(preprocessed)
            # print(prediction) # [[0.00533728 0.99466264]]

            if prediction[0,0] < prediction[0,1]:
                print('졸림 상태')
                MainWindow.sleep_cnt += 1
                # 졸린 상태가 30초간 지속되면 소리 
                if MainWindow.sleep_cnt % 100 == 0:
                    MainWindow.sleep_cnt = 1
                    print('30초간 졸고 있네요!!!')
                    beepsound()
                    # break ## 1번만 알람이 오면 프로그램을 정지 시킴 (반복을 원한다면, 주석으로 막기!)
            else:
                print('깨어있는 상태')
                MainWindow.sleep_cnt = 1

            # Convert the frame to a QImage
            image = QImage(frame_flipped, frame_flipped.shape[1], frame_flipped.shape[0], 
                           frame_flipped.shape[1] * 3, QImage.Format_BGR888)

            # Convert the QImage to a QPixmap and set it as the label's pixmap
            pixmap = QPixmap.fromImage(image)
            self.label.setPixmap(pixmap)

    def closeEvent(self, event):
        # Release the camera and stop the timer when closing the window
        self.capture.release()
        self.timer.stop()
        super().closeEvent(event)

if __name__ == '__main__':
    app = QApplication([])
    window = MainWindow()
    window.show()
    app.exec_()